In [321]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [322]:
lending_topredict = pd.read_csv("lending_topredict.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [323]:
lending_train = pd.read_csv("lending_train.csv")

### (0) DATA CLEANING
lending_train["months_since_last_delinq"] = lending_train["months_since_last_delinq"].fillna(0) # fill months_since_last_delinq nans with 0
# id is a non-random tag that offers no help
# zipcode aka location won't help too much
# extended reason too many things to consider
# get rid of employment column, 298340 unique values, too big of a categorical variable for ML training
# print(len(np.unique(lending_train["employment"])))
lending_train = lending_train.drop(["ID", "state", "zipcode"], axis=1)
lending_train["employment_length"] = lending_train["employment_length"].fillna(0)
lending_train["any_tax_liens"] = lending_train["any_tax_liens"].fillna(0)
lending_train["debt_to_income_ratio"] = lending_train["debt_to_income_ratio"].fillna(0)
lending_train["public_bankruptcies"] = lending_train["public_bankruptcies"].fillna(0)
lending_train["fico_inquired_last_6mths"] = lending_train["fico_inquired_last_6mths"].fillna(0)

# total_revolving_limit and revolving_balance have correlation of 0.8, so let's drop total_revolving_limit, which has null values
#lending_train.corr()
lending_train = lending_train.drop(["total_revolving_limit"], axis=1)

In [342]:
pd.read_csv("lending_train.csv").iloc[0,:]

ID                                           0
requested_amnt                         32000.0
loan_duration                        60 months
employment                                 SVP
employment_length                      4 years
race                                         W
reason_for_loan             debt_consolidation
extended_reason             Debt consolidation
annual_income                         250000.0
debt_to_income_ratio                     16.35
employment_verified                   Verified
public_bankruptcies                        0.0
zipcode                                  333xx
state                                       FL
home_ownership_status                     RENT
delinquency_last_2yrs                      0.0
fico_score_range_low                     775.0
fico_score_range_high                    779.0
fico_inquired_last_6mths                   0.0
months_since_last_delinq                   NaN
revolving_balance                      22480.0
total_revolvi

In [324]:
lending_topredict = pd.read_csv("lending_topredict.csv")

### (0) DATA CLEANING
lending_topredict["months_since_last_delinq"] = lending_topredict["months_since_last_delinq"].fillna(0) # fill months_since_last_delinq nans with 0
# id is a non-random tag that offers no help
# zipcode aka location won't help too much
# extended reason too many things to consider
# get rid of employment column, 298340 unique values, too big of a categorical variable for ML training
# print(len(np.unique(lending_train["employment"])))
lending_topredict = lending_topredict.drop(["ID", "state", "zipcode"], axis=1)
lending_topredict["employment_length"] = lending_topredict["employment_length"].fillna(0)
lending_topredict["any_tax_liens"] = lending_topredict["any_tax_liens"].fillna(0)
lending_topredict["debt_to_income_ratio"] = lending_topredict["debt_to_income_ratio"].fillna(0)
lending_topredict["public_bankruptcies"] = lending_topredict["public_bankruptcies"].fillna(0)
lending_topredict["fico_inquired_last_6mths"] = lending_topredict["fico_inquired_last_6mths"].fillna(0)

# total_revolving_limit and revolving_balance have correlation of 0.8, so let's drop total_revolving_limit, which has null values
#lending_train.corr()
lending_topredict = lending_topredict.drop(["total_revolving_limit"], axis=1)

In [325]:
import category_encoders as ce

encoder = ce.CatBoostEncoder()
X_train_ce = encoder.fit_transform(lending_train.drop(["loan_paid"], axis=1), lending_train[["loan_paid"]])
X_eval_ce = encoder.transform(lending_topredict.drop(["loan_paid"], axis = 1))

In [326]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_ce_scaled = scaler.fit_transform(X_train_ce, lending_train["loan_paid"])
X_eval_ce_scaled = scaler.transform(X_eval_ce)

In [327]:
### (2) SVC
y = lending_train["loan_paid"]
X_train, X_test, y_train, y_test = train_test_split(X_train_ce_scaled, y, test_size=0.20)
model = SVC(kernel='linear')
# fit the model
model.fit(X_train, y_train)
# get importance

RandomForestClassifier()

In [328]:
from sklearn.model_selection import GridSearchCV

# Creating the hyperparameter grid 
tuned_parameters = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
                    }
clf = GridSearchCV(model, tuned_parameters, cv=5, scoring="accuracy")
clf.fit(X_train, y_train)
  
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(clf.best_params_))
print("Best score is {}".format(clf.best_score_))

ValueError: Invalid parameter C for estimator RandomForestClassifier(). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
for i,v in enumerate(model.coef_[0]):
    print('Feature: %s, Score: %.5f' % (X_eval_ce.columns[i],v))

In [329]:
#y_pred = model.predict(X_test)
from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(y_test, model.predict(X_test)))

0.1254106260673679


In [330]:
# prediction
res = model.predict(X_eval_ce_scaled)

In [332]:
res_df = pd.DataFrame(pd.read_csv("lending_topredict.csv")["ID"], columns=['ID'])
res_df["loan_paid"] = res
res_df.to_csv("Loan_ToSubmit.csv",index=False)